# Inheritance

In [ ]:
from abc import ABC, abstractmethod
from datetime import datetime
import random

# Abstract base class for all vehicles
class Vehicle(ABC):
    def __init__(self, make: str, model: str, year: int):
        self.make = make
        self.model = model
        self.year = year
        self.mileage = 0
        self.is_running = False
        self.serial_number = f"{make[:2]}{random.randint(10000, 99999)}"

    @abstractmethod
    def calculate_maintenance_cost(self) -> float:
        """Abstract method that must be implemented by all subclasses"""
        pass

    def start_engine(self) -> str:
        self.is_running = True
        return f"{self.make} {self.model} engine started"

    def stop_engine(self) -> str:
        self.is_running = False
        return f"{self.make} {self.model} engine stopped"

    def __str__(self) -> str:
        return f"{self.year} {self.make} {self.model} (SN: {self.serial_number})"



In [8]:
# Interface-like class for electric vehicles
class ElectricVehicleInterface(ABC):
    @abstractmethod
    def charge_battery(self) -> str:
        pass

    @abstractmethod
    def get_battery_range(self) -> int:
        pass


In [9]:

# Base class for land vehicles
class LandVehicle(Vehicle):
    def __init__(self, make: str, model: str, year: int, num_wheels: int):
        super().__init__(make, model, year)
        self.num_wheels = num_wheels
        self.last_service_date = datetime.now()

    def drive(self, distance: float) -> str:
        if self.is_running:
            self.mileage += distance
            return f"Drove {distance} miles. Total mileage: {self.mileage}"
        return "Start the engine first!"

    def calculate_maintenance_cost(self) -> float:
        # Base maintenance cost calculation
        base_cost = 100.0
        mileage_factor = self.mileage * 0.05
        age_factor = (datetime.now().year - self.year) * 50
        return base_cost + mileage_factor + age_factor


In [10]:

# Concrete class for cars with multiple inheritance
class ElectricCar(LandVehicle, ElectricVehicleInterface):
    def __init__(self, make: str, model: str, year: int, battery_capacity: float):
        super().__init__(make, model, year, num_wheels=4)
        self.battery_capacity = battery_capacity  # in kWh
        self.battery_level = 100.0  # percentage
        self.range_per_charge = 300  # miles

    def charge_battery(self) -> str:
        if self.battery_level < 100:
            self.battery_level = 100
            return "Battery fully charged"
        return "Battery already full"

    def get_battery_range(self) -> int:
        return int(self.range_per_charge * (self.battery_level / 100))

    # Override parent method
    def drive(self, distance: float) -> str:
        if not self.is_running:
            return "Start the engine first!"

        range_left = self.get_battery_range()
        if distance > range_left:
            return f"Insufficient range! Only {range_left} miles remaining"

        self.mileage += distance
        self.battery_level -= (distance / self.range_per_charge) * 100
        return f"Drove {distance} miles. Battery: {self.battery_level:.1f}%"

    # Override abstract method with specific implementation
    def calculate_maintenance_cost(self) -> float:
        base_cost = super().calculate_maintenance_cost()
        battery_cost = self.battery_capacity * 10  # Additional cost for battery maintenance
        return base_cost + battery_cost


In [11]:

# Concrete class for motorcycles
class Motorcycle(LandVehicle):
    def __init__(self, make: str, model: str, year: int, has_sidecar: bool = False):
        super().__init__(make, model, year, num_wheels=3 if has_sidecar else 2)
        self.has_sidecar = has_sidecar

    # Override parent method with specific behavior
    def calculate_maintenance_cost(self) -> float:
        base_cost = super().calculate_maintenance_cost()
        sidecar_cost = 150.0 if self.has_sidecar else 0
        return base_cost * 0.8 + sidecar_cost  # Motorcycles generally cheaper to maintain


In [12]:

# Demonstration
def main():
    # Create instances
    tesla = ElectricCar("Tesla", "Model 3", 2022, battery_capacity=75.0)
    harley = Motorcycle("Harley-Davidson", "Sportster", 2021, has_sidecar=True)

    # Test ElectricCar
    print(tesla)
    print(tesla.start_engine())
    print(tesla.drive(100))
    print(f"Range remaining: {tesla.get_battery_range()} miles")
    print(f"Maintenance cost: ${tesla.calculate_maintenance_cost():.2f}")
    print(tesla.charge_battery())
    print()

    # Test Motorcycle
    print(harley)
    print(harley.start_engine())
    print(harley.drive(50))
    print(f"Maintenance cost: ${harley.calculate_maintenance_cost():.2f}")

if __name__ == "__main__":
    main()

2022 Tesla Model 3 (SN: Te92521)
Tesla Model 3 engine started
Drove 100 miles. Battery: 66.7%
Range remaining: 200 miles
Maintenance cost: $1005.00
Battery fully charged

2021 Harley-Davidson Sportster (SN: Ha62400)
Harley-Davidson Sportster engine started
Drove 50 miles. Total mileage: 50
Maintenance cost: $392.00
